This project will analyze what weather factors contribute to atlanta traffic crashes. Moreover, can we predict whether or not the accident happened at a traffic signal. This analysis is being conducted because I was hired for a job in atlanta. I come from a very small town with a yellow flashing intersection so this analysis will be used to help me determine how my driving habits need to change. 

In [1]:
#regular imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#model imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
pd.set_option('display.max_columns', None)


In [3]:
df = pd.read_csv('traffic_accidents_atlanta.csv')


In [4]:
df.describe()

,Unnamed: 0,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Number,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
count,1.185400e+04,11854.000000,11854.000000,11854.000000,11854.000000,11854.000000,11854.000000,960.000000,11820.000000,6564.000000,11804.000000,11828.000000,11819.000000,10716.000000,6731.000000
mean,8.416614e+05,2.489961,33.789393,-84.397831,33.789629,-84.397464,0.566292,1971.509375,62.519932,57.957252,71.216876,29.572356,8.925665,7.283147,0.017501
std,4.930253e+05,0.719280,0.232366,0.109199,0.232324,0.109679,0.676944,2498.971731,15.515560,18.284380,21.157897,0.553521,2.411730,4.704508,0.072685
min,3.259600e+04,1.000000,33.569125,-89.230820,33.563020,-89.221158,0.000000,1.000000,6.300000,-3.000000,11.000000,28.320000,0.100000,0.000000,0.000000
25%,3.720768e+05,2.000000,33.720463,-84.449995,33.726242,-84.450518,0.165250,279.000000,51.100000,42.000000,54.000000,29.030000,10.000000,4.600000,0.000000
50%,9.018595e+05,2.000000,33.768099,-84.392039,33.767740,-84.391847,0.398000,1113.000000,64.000000,61.000000,75.000000,29.780000,10.000000,6.900000,0.000000
75%,1.301238e+06,3.000000,33.880251,-84.357950,33.881150,-84.357960,0.689000,3183.750000,73.900000,72.000000,90.000000,30.080000,10.000000,10.000000,0.000000
max,1.515742e+06,4.000000,44.899080,-84.031240,44.899080,-84.031240,10.931000,29599.000000,98.000000,98.000000,100.000000,30.750000,10.000000,33.400000,1.400000


In [5]:
df.replace('', None, inplace=True)

In [6]:
df.isna().sum()

Unnamed: 0                   0
ID                           0
Severity                     0
Start_Time                   0
End_Time                     0
Start_Lat                    0
Start_Lng                    0
End_Lat                      0
End_Lng                      0
Distance(mi)                 0
Description                  0
Number                   10894
Street                       0
Side                         0
City                         0
County                       0
State                        0
Zipcode                      0
Country                      0
Timezone                     0
Airport_Code                 0
Weather_Timestamp           25
Temperature(F)              34
Wind_Chill(F)             5290
Humidity(%)                 50
Pressure(in)                26
Visibility(mi)              35
Wind_Direction              47
Wind_Speed(mph)           1138
Precipitation(in)         5123
Weather_Condition           32
Amenity                      0
Bump    

In [7]:
df = df.drop(columns=['Number', 'Wind_Chill(F)'])
print(df.isna().sum())
cleanedDF = df.dropna(axis=0)#, how='all')


Unnamed: 0                  0
ID                          0
Severity                    0
Start_Time                  0
End_Time                    0
Start_Lat                   0
Start_Lng                   0
End_Lat                     0
End_Lng                     0
Distance(mi)                0
Description                 0
Street                      0
Side                        0
City                        0
County                      0
State                       0
Zipcode                     0
Country                     0
Timezone                    0
Airport_Code                0
Weather_Timestamp          25
Temperature(F)             34
Humidity(%)                50
Pressure(in)               26
Visibility(mi)             35
Wind_Direction             47
Wind_Speed(mph)          1138
Precipitation(in)        5123
Weather_Condition          32
Amenity                     0
Bump                        0
Crossing                    0
Give_Way                    0
Junction  

In [8]:
print(cleanedDF.isna().sum())

Unnamed: 0               0
ID                       0
Severity                 0
Start_Time               0
End_Time                 0
Start_Lat                0
Start_Lng                0
End_Lat                  0
End_Lng                  0
Distance(mi)             0
Description              0
Street                   0
Side                     0
City                     0
County                   0
State                    0
Zipcode                  0
Country                  0
Timezone                 0
Airport_Code             0
Weather_Timestamp        0
Temperature(F)           0
Humidity(%)              0
Pressure(in)             0
Visibility(mi)           0
Wind_Direction           0
Wind_Speed(mph)          0
Precipitation(in)        0
Weather_Condition        0
Amenity                  0
Bump                     0
Crossing                 0
Give_Way                 0
Junction                 0
No_Exit                  0
Railway                  0
Roundabout               0
S

In [9]:
cleanedDF.head(15)

,Unnamed: 0,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,32596,A-2749196,2,2016-11-30 17:39:56,2016-11-30 23:39:56,33.741530,-84.414340,33.746580,-84.430340,0.983,Between Ashby St/Exit 55 and Langhorn St/Exit ...,GA-402 W,R,Atlanta,Fulton,GA,30310,US,US/Eastern,KATL,2016-11-30 17:46:00,62.6,100.0,29.80,8.0,West,9.2,0.00,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Day,Day,Day
17,33663,A-2750263,4,2016-12-03 22:08:28,2016-12-04 04:08:28,33.744630,-84.399230,33.744880,-84.397030,0.128,Ramp closed to I-75/I-85 James Wendell George ...,GA-402 E,R,Atlanta,Fulton,GA,30312,US,US/Eastern,KATL,2016-12-03 21:52:00,46.9,90.0,30.19,10.0,East,8.1,0.00,Light Rain,False,False,False,False,True,False,False,False,False,False,False,False,False,Night,Night,Night,Night
18,33664,A-2750264,4,2016-12-03 22:08:28,2016-12-04 04:08:28,33.744880,-84.397030,33.746370,-84.387650,0.549,Ramp closed to I-75/I-85 Northbound - Road clo...,Rawson St SW,L,Atlanta,Fulton,GA,30312,US,US/Eastern,KATL,2016-12-03 21:52:00,46.9,90.0,30.19,10.0,East,8.1,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
19,33667,A-2750267,4,2016-12-03 22:08:28,2016-12-04 04:08:28,33.746370,-84.387650,33.748570,-84.383970,0.260,Ramp closed to Exit 57 - Road closed due to ac...,Capitol Ave SE,R,Atlanta,Fulton,GA,30312,US,US/Eastern,KATL,2016-12-03 21:52:00,46.9,90.0,30.19,10.0,East,8.1,0.00,Light Rain,False,False,True,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
20,33676,A-2750276,3,2016-12-04 00:10:43,2016-12-04 06:10:43,33.804140,-84.410610,33.807200,-84.416150,0.382,At Howell Mill Rd/Exit 252 - Accident.,I-75 N,R,Atlanta,Fulton,GA,30318,US,US/Eastern,KFTY,2016-12-04 00:53:00,46.0,89.0,30.17,10.0,ENE,5.8,0.00,Overcast,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
25,33697,A-2750297,4,2016-12-04 05:43:26,2016-12-04 11:43:26,33.639070,-84.378110,33.632570,-84.407430,1.745,Closed between US-19/US-41/Old Dixie Rd/Exit 5...,GA-407 W,R,Atlanta,Clayton,GA,30354,US,US/Eastern,KATL,2016-12-04 05:52:00,46.0,89.0,30.13,10.0,East,12.7,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
26,33705,A-2750305,3,2016-12-04 08:15:45,2016-12-04 14:15:45,33.685880,-84.499860,33.693990,-84.499980,0.560,At GA-166/Lakewood Fwy/Exit 5 - Accident.,I-285 N,R,Atlanta,Fulton,GA,30331,US,US/Eastern,KFTY,2016-12-04 07:53:00,45.0,86.0,30.16,5.0,East,8.1,0.09,Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day
27,33724,A-2750324,3,2016-12-04 10:18:47,2016-12-04 16:18:47,33.758350,-84.493870,33.751070,-84.496590,0.527,At GA-139/ML King Jr Dr/Exit 9 - Accident.,GA-407 S,R,Atlanta,Fulton,GA,30331,US,US/Eastern,KFTY,2016-12-04 10:32:00,45.0,86.0,30.16,3.0,ENE,8.1,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day
28,33731,A-2750331,3,2016-12-04 11:41:09,2016-12-04 17:41:09,33.909980,-84.283070,33.916190,-84.289450,0.564,At GA-141/Exit 31 - Accident.,GA-407 W,R,Atlanta,DeKalb,GA,30360,US,US/Eastern,KPDK,2016-12-04 11:53:00,44.1,89.0,30.16,1.8,East,6.9,0.10,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day
29,33733,A-2750333,3,2016-12-04 12:29:47,2016-12-04 18:29:47,33.889060,-84.255440,33.899142,-84.268463,1.021,Between I-85/Exit 33 (East) and US-

In [10]:
# remove irrelevant columns based upon research question
cleanedDF = cleanedDF.drop(columns= ['Unnamed: 0', 'ID', 'Severity', 'Start_Time', 'End_Time', 
                       'Start_Lat', 'End_Lat', 'Start_Lng', 'End_Lng', 'Description', 'Street', 
                       'City', 'County', 'State', 'Zipcode', 'Timezone', 'Airport_Code', 
                       'Weather_Timestamp', 'Astronomical_Twilight', 'Nautical_Twilight']).reset_index()

In [11]:
print(cleanedDF.isna().sum())

index                0
Distance(mi)         0
Side                 0
Country              0
Temperature(F)       0
Humidity(%)          0
Pressure(in)         0
Visibility(mi)       0
Wind_Direction       0
Wind_Speed(mph)      0
Precipitation(in)    0
Weather_Condition    0
Amenity              0
Bump                 0
Crossing             0
Give_Way             0
Junction             0
No_Exit              0
Railway              0
Roundabout           0
Station              0
Stop                 0
Traffic_Calming      0
Traffic_Signal       0
Turning_Loop         0
Sunrise_Sunset       0
Civil_Twilight       0
dtype: int64


In [12]:
cleanedDF.tail()

,index,Distance(mi),Side,Country,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight
6594,11849,0.345,R,US,91.0,45.0,28.86,10.0,SSW,7.0,0.0,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day
6595,11850,0.569,R,US,79.0,69.0,29.14,10.0,CALM,0.0,0.0,Fair,False,False,False,False,True,False,False,False,False,False,False,False,False,Night,Night
6596,11851,0.665,R,US,85.0,63.0,28.92,10.0,NNW,5.0,0.0,Partly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day
6597,11852,0.306,R,US,80.0,74.0,28.94,10.0,N,5.0,0.0,Fair,False,False,False,False,True,False,False,False,False,False,False,False,False,Night,Night
6598,11853,0.389,R,US,80.0,74.0,28.94,10.0,N,5.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Day


In [13]:
categoricalCols = []
numericalCols = []
for col in cleanedDF.columns:
    if cleanedDF[col].dtype == 'int64' or cleanedDF[col].dtype == 'float':
        numericalCols.append(col)
    else:
        categoricalCols.append(col)

categoricalCols

['Side',
 'Country',
 'Wind_Direction',
 'Weather_Condition',
 'Amenity',
 'Bump',
 'Crossing',
 'Give_Way',
 'Junction',
 'No_Exit',
 'Railway',
 'Roundabout',
 'Station',
 'Stop',
 'Traffic_Calming',
 'Traffic_Signal',
 'Turning_Loop',
 'Sunrise_Sunset',
 'Civil_Twilight']

In [14]:
numericalCols

['index',
 'Distance(mi)',
 'Temperature(F)',
 'Humidity(%)',
 'Pressure(in)',
 'Visibility(mi)',
 'Wind_Speed(mph)',
 'Precipitation(in)']

In [15]:
# for col in categoricalCols:
#     cleanedDF[col].value_counts().plot(kind='bar', title=col)
#     plt.xlabel(col)
#     plt.ylabel("Count")
#     plt.show()

In [16]:
# for col in numericalCols:
#     plt.figure(figsize=(10,6))
#     sns.boxplot(x=cleanedDF[col])
#     plt.title(col)
#     plt.show()

After examining outliers, I have determined that I want to include this information to accont for things such as weather anomalies

In [17]:
from sklearn.preprocessing import OneHotEncoder


#instantiate and fit the one hot encoder
encoder = OneHotEncoder(sparse=False)
encoderArray = encoder.fit_transform(cleanedDF[categoricalCols])
colNames = encoder.get_feature_names_out()

#fix the shape
colNames = np.array(colNames).ravel()

#get the dataframe of the 
oneHotDF = pd.DataFrame(encoderArray, columns=colNames)

oneHotDF.head()

/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Side_L,Side_R,Country_US,Wind_Direction_CALM,Wind_Direction_E,Wind_Direction_ENE,Wind_Direction_ESE,Wind_Direction_East,Wind_Direction_N,Wind_Direction_NE,Wind_Direction_NNE,Wind_Direction_NNW,Wind_Direction_NW,Wind_Direction_North,Wind_Direction_S,Wind_Direction_SE,Wind_Direction_SSE,Wind_Direction_SSW,Wind_Direction_SW,Wind_Direction_South,Wind_Direction_VAR,Wind_Direction_Variable,Wind_Direction_W,Wind_Direction_WNW,Wind_Direction_WSW,Wind_Direction_West,Weather_Condition_Clear,Weather_Condition_Cloudy,Weather_Condition_Cloudy / Windy,Weather_Condition_Drizzle and Fog,Weather_Condition_Fair,Weather_Condition_Fair / Windy,Weather_Condition_Fog,Weather_Condition_Haze,Weather_Condition_Heavy Rain,Weather_Condition_Heavy Rain / Windy,Weather_Condition_Heavy T-Storm,Weather_Condition_Heavy Thunderstorms and Rain,Weather_Condition_Light Drizzle,Weather_Condition_Light Drizzle / Windy,Weather_Condition_Light Ice Pellets,Weather_Condition_Light Rain,Weather_Condition_Light Rain / Windy,Weather_Condition_Light Rain with Thunder,Weather_Condition_Light Snow,Weather_Condition_Light Snow / Windy,Weather_Condition_Light Thunderstorms and Rain,Weather_Condition_Mostly Cloudy,Weather_Condition_Mostly Cloudy / Windy,Weather_Condition_Overcast,Weather_Condition_Partly Cloudy,Weather_Condition_Partly Cloudy / Windy,Weather_Condition_Rain,Weather_Condition_Rain / Windy,Weather_Condition_Rain Shower,Weather_Condition_Scattered Clouds,Weather_Condition_T-Storm,Weather_Condition_Thunder,Weather_Condition_Thunder in the Vicinity,Weather_Condition_Thunderstorm,Weather_Condition_Thunderstorms and Rain,Weather_Condition_Wintry Mix / Windy,Amenity_False,Amenity_True,Bump_False,Crossing_False,Crossing_True,Give_Way_False,Junction_False,Junction_True,No_Exit_False,No_Exit_True,Railway_False,Railway_True,Roundabout_False,Station_False,Station_True,Stop_False,Stop_True,Traffic_Calming_False,Traffic_Signal_False,Traffic_Signal_True,Turning_Loop_False,Sunrise_Sunset_Day,Sunrise_Sunset_Night,Civil_Twilight_Day,Civil_Twilight_Night
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [18]:
cleanedDF.shape

(6599, 27)

In [19]:
oneHotDF.shape

(6599, 87)

In [20]:
print(cleanedDF.isna().sum())

index                0
Distance(mi)         0
Side                 0
Country              0
Temperature(F)       0
Humidity(%)          0
Pressure(in)         0
Visibility(mi)       0
Wind_Direction       0
Wind_Speed(mph)      0
Precipitation(in)    0
Weather_Condition    0
Amenity              0
Bump                 0
Crossing             0
Give_Way             0
Junction             0
No_Exit              0
Railway              0
Roundabout           0
Station              0
Stop                 0
Traffic_Calming      0
Traffic_Signal       0
Turning_Loop         0
Sunrise_Sunset       0
Civil_Twilight       0
dtype: int64


In [21]:
print(oneHotDF.isna().sum())

Side_L                  0
Side_R                  0
Country_US              0
Wind_Direction_CALM     0
Wind_Direction_E        0
                       ..
Turning_Loop_False      0
Sunrise_Sunset_Day      0
Sunrise_Sunset_Night    0
Civil_Twilight_Day      0
Civil_Twilight_Night    0
Length: 87, dtype: int64


In [22]:
cleanedDF = cleanedDF.drop(columns=categoricalCols, axis=1)

cleanedDF.shape

(6599, 8)

In [23]:
oneHotDF.shape

(6599, 87)

In [24]:
newDF = pd.concat([cleanedDF, oneHotDF], axis=1)

newDF.tail()
#newDF.drop(newDF.tail(376).index, inplace = True)

,index,Distance(mi),Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Side_L,Side_R,Country_US,Wind_Direction_CALM,Wind_Direction_E,Wind_Direction_ENE,Wind_Direction_ESE,Wind_Direction_East,Wind_Direction_N,Wind_Direction_NE,Wind_Direction_NNE,Wind_Direction_NNW,Wind_Direction_NW,Wind_Direction_North,Wind_Direction_S,Wind_Direction_SE,Wind_Direction_SSE,Wind_Direction_SSW,Wind_Direction_SW,Wind_Direction_South,Wind_Direction_VAR,Wind_Direction_Variable,Wind_Direction_W,Wind_Direction_WNW,Wind_Direction_WSW,Wind_Direction_West,Weather_Condition_Clear,Weather_Condition_Cloudy,Weather_Condition_Cloudy / Windy,Weather_Condition_Drizzle and Fog,Weather_Condition_Fair,Weather_Condition_Fair / Windy,Weather_Condition_Fog,Weather_Condition_Haze,Weather_Condition_Heavy Rain,Weather_Condition_Heavy Rain / Windy,Weather_Condition_Heavy T-Storm,Weather_Condition_Heavy Thunderstorms and Rain,Weather_Condition_Light Drizzle,Weather_Condition_Light Drizzle / Windy,Weather_Condition_Light Ice Pellets,Weather_Condition_Light Rain,Weather_Condition_Light Rain / Windy,Weather_Condition_Light Rain with Thunder,Weather_Condition_Light Snow,Weather_Condition_Light Snow / Windy,Weather_Condition_Light Thunderstorms and Rain,Weather_Condition_Mostly Cloudy,Weather_Condition_Mostly Cloudy / Windy,Weather_Condition_Overcast,Weather_Condition_Partly Cloudy,Weather_Condition_Partly Cloudy / Windy,Weather_Condition_Rain,Weather_Condition_Rain / Windy,Weather_Condition_Rain Shower,Weather_Condition_Scattered Clouds,Weather_Condition_T-Storm,Weather_Condition_Thunder,Weather_Condition_Thunder in the Vicinity,Weather_Condition_Thunderstorm,Weather_Condition_Thunderstorms and Rain,Weather_Condition_Wintry Mix / Windy,Amenity_False,Amenity_True,Bump_False,Crossing_False,Crossing_True,Give_Way_False,Junction_False,Junction_True,No_Exit_False,No_Exit_True,Railway_False,Railway_True,Roundabout_False,Station_False,Station_True,Stop_False,Stop_True,Traffic_Calming_False,Traffic_Signal_False,Traffic_Signal_True,Turning_Loop_False,Sunrise_Sunset_Day,Sunrise_Sunset_Night,Civil_Twilight_Day,Civil_Twilight_Night
6594,11849,0.345,91.0,45.0,28.86,10.0,7.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
6595,11850,0.569,79.0,69.0,29.14,10.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
6596,11851,0.665,85.0,63.0,28.92,10.0,5.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
6597,11852,0.306,80.0,74.0,28.94,10.0,5.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
6598,11853,0.389,80.0,74.0,28.94,10.0,5.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [25]:
print(newDF.isna().sum())

index                   0
Distance(mi)            0
Temperature(F)          0
Humidity(%)             0
Pressure(in)            0
                       ..
Turning_Loop_False      0
Sunrise_Sunset_Day      0
Sunrise_Sunset_Night    0
Civil_Twilight_Day      0
Civil_Twilight_Night    0
Length: 95, dtype: int64


In [26]:
newDF.shape

(6599, 95)

In [27]:
yDF = newDF['Precipitation(in)']
xDF = newDF.drop(columns=['Precipitation(in)'], axis=1)

print(xDF.columns)
xDF.head()

Index(['index', 'Distance(mi)', 'Temperature(F)', 'Humidity(%)',
       'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Side_L', 'Side_R',
       'Country_US', 'Wind_Direction_CALM', 'Wind_Direction_E',
       'Wind_Direction_ENE', 'Wind_Direction_ESE', 'Wind_Direction_East',
       'Wind_Direction_N', 'Wind_Direction_NE', 'Wind_Direction_NNE',
       'Wind_Direction_NNW', 'Wind_Direction_NW', 'Wind_Direction_North',
       'Wind_Direction_S', 'Wind_Direction_SE', 'Wind_Direction_SSE',
       'Wind_Direction_SSW', 'Wind_Direction_SW', 'Wind_Direction_South',
       'Wind_Direction_VAR', 'Wind_Direction_Variable', 'Wind_Direction_W',
       'Wind_Direction_WNW', 'Wind_Direction_WSW', 'Wind_Direction_West',
       'Weather_Condition_Clear', 'Weather_Condition_Cloudy',
       'Weather_Condition_Cloudy / Windy', 'Weather_Condition_Drizzle and Fog',
       'Weather_Condition_Fair', 'Weather_Condition_Fair / Windy',
       'Weather_Condition_Fog', 'Weather_Condition_Haze',
       'Weat

,index,Distance(mi),Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Side_L,Side_R,Country_US,Wind_Direction_CALM,Wind_Direction_E,Wind_Direction_ENE,Wind_Direction_ESE,Wind_Direction_East,Wind_Direction_N,Wind_Direction_NE,Wind_Direction_NNE,Wind_Direction_NNW,Wind_Direction_NW,Wind_Direction_North,Wind_Direction_S,Wind_Direction_SE,Wind_Direction_SSE,Wind_Direction_SSW,Wind_Direction_SW,Wind_Direction_South,Wind_Direction_VAR,Wind_Direction_Variable,Wind_Direction_W,Wind_Direction_WNW,Wind_Direction_WSW,Wind_Direction_West,Weather_Condition_Clear,Weather_Condition_Cloudy,Weather_Condition_Cloudy / Windy,Weather_Condition_Drizzle and Fog,Weather_Condition_Fair,Weather_Condition_Fair / Windy,Weather_Condition_Fog,Weather_Condition_Haze,Weather_Condition_Heavy Rain,Weather_Condition_Heavy Rain / Windy,Weather_Condition_Heavy T-Storm,Weather_Condition_Heavy Thunderstorms and Rain,Weather_Condition_Light Drizzle,Weather_Condition_Light Drizzle / Windy,Weather_Condition_Light Ice Pellets,Weather_Condition_Light Rain,Weather_Condition_Light Rain / Windy,Weather_Condition_Light Rain with Thunder,Weather_Condition_Light Snow,Weather_Condition_Light Snow / Windy,Weather_Condition_Light Thunderstorms and Rain,Weather_Condition_Mostly Cloudy,Weather_Condition_Mostly Cloudy / Windy,Weather_Condition_Overcast,Weather_Condition_Partly Cloudy,Weather_Condition_Partly Cloudy / Windy,Weather_Condition_Rain,Weather_Condition_Rain / Windy,Weather_Condition_Rain Shower,Weather_Condition_Scattered Clouds,Weather_Condition_T-Storm,Weather_Condition_Thunder,Weather_Condition_Thunder in the Vicinity,Weather_Condition_Thunderstorm,Weather_Condition_Thunderstorms and Rain,Weather_Condition_Wintry Mix / Windy,Amenity_False,Amenity_True,Bump_False,Crossing_False,Crossing_True,Give_Way_False,Junction_False,Junction_True,No_Exit_False,No_Exit_True,Railway_False,Railway_True,Roundabout_False,Station_False,Station_True,Stop_False,Stop_True,Traffic_Calming_False,Traffic_Signal_False,Traffic_Signal_True,Turning_Loop_False,Sunrise_Sunset_Day,Sunrise_Sunset_Night,Civil_Twilight_Day,Civil_Twilight_Night
0,0,0.983,62.6,100.0,29.80,8.0,9.2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
1,17,0.128,46.9,90.0,30.19,10.0,8.1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
2,18,0.549,46.9,90.0,30.19,10.0,8.1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
3,19,0.260,46.9,90.0,30.19,10.0,8.1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,20,0.382,46.0,89.0,30.17,10.0,5.8,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.

In [28]:
from sklearn.model_selection import train_test_split

xTrain, xTest, yTrain, yTest = train_test_split(
    xDF, yDF,
    test_size= 0.25,
    shuffle= True)

In [29]:
pd.DataFrame(yDF)

,Precipitation(in)
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
6594,0.0
6595,0.0
6596,0.0
6597,0.0


In [30]:
from sklearn.preprocessing import StandardScaler

xScaler = StandardScaler()
xColNames = xTrain.columns.values.tolist()
# train the scaler and apply it to the training set
xTrainScaled = xScaler.fit_transform(xTrain[xColNames])

In [31]:
from sklearn.model_selection import GridSearchCV

In [32]:
from sklearn.linear_model import LinearRegression

parameters = {
              "fit_intercept": [True, False],
              "positive" : [True, False]
             }
gridSearch = GridSearchCV(estimator = LinearRegression(),  
                           param_grid = parameters,
                           scoring = 'recall',
                           cv = 5,
                           verbose=0)


gridSearch.fit(xTrainScaled, yTrain) 

linearParams = gridSearch.best_params_

linearParams

/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
           ^^^^^^^^^^^^
  File "/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classific

/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
           ^^^^^^^^^^^^
  File "/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classific

{'fit_intercept': True, 'positive': True}

In [50]:
from sklearn.ensemble import AdaBoostRegressor

parameters = {
              'n_estimators':[10, 50, 100, 200],
              'learning_rate':[0.1, 0.5, 1, 2]}
gridSearch = GridSearchCV(estimator = AdaBoostRegressor(),  
                           param_grid = parameters,
                           scoring = 'recall')


gridSearch.fit(xTrainScaled, yTrain) 

adaBoostParams = gridSearch.best_params_

adaBoostParams

/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
           ^^^^^^^^^^^^
  File "/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classific

{'learning_rate': 0.1, 'n_estimators': 10}

In [52]:
from sklearn.ensemble import RandomForestRegressor

parameters = {
              'n_estimators':[10, 50, 100, 200],
              'max_depth':[None, 5, 10, 15],
              'min_samples_leaf':[1, 5, 10]}
gridSearch = GridSearchCV(estimator = RandomForestRegressor(),  
                           param_grid = parameters,
                           scoring = 'r2')


gridSearch.fit(xTrainScaled, yTrain) 

randomForestParams = gridSearch.best_params_

randomForestParams

{'max_depth': 15, 'min_samples_leaf': 1, 'n_estimators': 200}

In [36]:
from sklearn.linear_model import Lasso

parameters = {
              'alpha':[0.1, 0.5, 1, 2]}
gridSearch = GridSearchCV(estimator = Lasso(),  
                           param_grid = parameters,
                           scoring = 'r2')


gridSearch.fit(xTrainScaled, yTrain) 

lassoParams = gridSearch.best_params_

lassoParams

{'alpha': 0.1}

In [37]:
from sklearn.linear_model import Ridge

parameters = {
              'alpha':[0.1, 0.5, 1, 2]}
gridSearch = GridSearchCV(estimator = Ridge(),  
                           param_grid = parameters,
                           scoring = 'r2')


gridSearch.fit(xTrainScaled, yTrain) 

ridgeParams = gridSearch.best_params_

ridgeParams

{'alpha': 2}

In [40]:
from sklearn.svm import LinearSVR

parameters = {
              'C':[0.1, 0.5, 1, 2]}
gridSearch = GridSearchCV(estimator = LinearSVR(),  
                           param_grid = parameters,
                           scoring = 'r2')


gridSearch.fit(xTrainScaled, yTrain) 

linearSVRParams = gridSearch.best_params_

linearSVRParams

/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/s

{'C': 0.1}

In [41]:
from sklearn.linear_model import ElasticNet

parameters = {
              'alpha':[0.1, 0.5, 1, 2],
              'l1_ratio':[0.1, 0.5, 0.9]}
gridSearch = GridSearchCV(estimator = ElasticNet(),  
                           param_grid = parameters,
                           scoring = 'r2')


gridSearch.fit(xTrainScaled, yTrain) 

elasticNetParams = gridSearch.best_params_

elasticNetParams

{'alpha': 0.1, 'l1_ratio': 0.1}

In [42]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import r2_score, mean_squared_error

In [43]:

cv = KFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {'fit_intercept': [True, False], 'normalize': [True, False]}

r2Scores = []
rmseScores = []

for i, (trainIndex, testIndex) in enumerate(cv.split(xDF, yDF)):
    #training and validation sets
    #convert indices to list
    trainIndex = trainIndex.tolist()
    testIndex = testIndex.tolist()

    print(trainIndex)

    #split the data into training and testing sets for this fold
    xTrain, xTest = xDF.iloc[trainIndex], xDF.iloc[testIndex]
    yTrain, yTest = yDF.iloc[trainIndex], yDF.iloc[testIndex]
     

    #feature scaling 
    ### feature scaling
    xScaler = StandardScaler()
    xColNames = xTrain.columns.values.tolist()
    # train the scaler and apply it to the training set
    xTrainScaled = xScaler.fit_transform(xTrain[xColNames])
    # apply the scaling to the testing set
    xTestScaled = xScaler.transform(xTest[xColNames])

    ### model training
    # instantiate the model
    clf = LinearRegression(**linearParams)
    # Train the classifier on the training data
    clf.fit(xTrainScaled, yTrain)
    
    ### model prediction and evaluation
    # Make predictions on the test data
    yPred = clf.predict(xTestScaled)
    
    # Calculate metrics and store them
    r2Score = r2_score(yTest, yPred)
    r2Scores.append(r2Score)

    rmseScore = mean_squared_error(yTest, yPred, squared=False)
    rmseScores.append(rmseScore)

    print(f"Completed Fold {i}")

### Calculate the mean scores across all folds
avgR2Score = sum(r2Scores) / len(r2Scores)
print("Mean r squared score:", avgR2Score)

avgRMSE = sum(rmseScores) / len(rmseScores)
print("Mean rmse:", avgRMSE)



[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 16, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 32, 34, 35, 36, 38, 39, 40, 42, 43, 46, 47, 48, 49, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 81, 82, 83, 85, 86, 87, 89, 90, 92, 94, 95, 97, 98, 99, 100, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 123, 125, 126, 127, 128, 129, 130, 131, 133, 134, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 147, 148, 150, 151, 152, 153, 154, 155, 158, 159, 160, 161, 162, 163, 164, 165, 166, 169, 170, 171, 173, 174, 175, 176, 178, 180, 182, 183, 184, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197, 200, 201, 202, 203, 204, 205, 206, 207, 209, 210, 211, 212, 213, 214, 215, 218, 219, 220, 222, 223, 224, 225, 226, 229, 232, 234, 235, 236, 237, 238, 241, 242, 243, 244, 246, 249, 250, 251, 252, 253, 255, 256, 258, 260, 261, 262, 264, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 280, 281, 282

In [44]:

cv = KFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {'fit_intercept': [True, False], 'normalize': [True, False]}

r2Scores = []
rmseScores = []

for i, (trainIndex, testIndex) in enumerate(cv.split(xDF, yDF)):
    #training and validation sets
    #convert indices to list
    trainIndex = trainIndex.tolist()
    testIndex = testIndex.tolist()

    print(trainIndex)

    #split the data into training and testing sets for this fold
    xTrain, xTest = xDF.iloc[trainIndex], xDF.iloc[testIndex]
    yTrain, yTest = yDF.iloc[trainIndex], yDF.iloc[testIndex]
     

    #feature scaling 
    ### feature scaling
    xScaler = StandardScaler()
    xColNames = xTrain.columns.values.tolist()
    # train the scaler and apply it to the training set
    xTrainScaled = xScaler.fit_transform(xTrain[xColNames])
    # apply the scaling to the testing set
    xTestScaled = xScaler.transform(xTest[xColNames])

    ### model training
    # instantiate the model
    clf = LinearSVR(**linearSVRParams)
    # Train the classifier on the training data
    clf.fit(xTrainScaled, yTrain)
    
    ### model prediction and evaluation
    # Make predictions on the test data
    yPred = clf.predict(xTestScaled)
    
    # Calculate metrics and store them
    r2Score = r2_score(yTest, yPred)
    r2Scores.append(r2Score)

    rmseScore = mean_squared_error(yTest, yPred, squared=False)
    rmseScores.append(rmseScore)

    print(f"Completed Fold {i}")

### Calculate the mean scores across all folds
avgR2Score = sum(r2Scores) / len(r2Scores)
print("Mean r squared score:", avgR2Score)

avgRMSE = sum(rmseScores) / len(rmseScores)
print("Mean rmse:", avgRMSE)



[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 16, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 32, 34, 35, 36, 38, 39, 40, 42, 43, 46, 47, 48, 49, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 81, 82, 83, 85, 86, 87, 89, 90, 92, 94, 95, 97, 98, 99, 100, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 123, 125, 126, 127, 128, 129, 130, 131, 133, 134, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 147, 148, 150, 151, 152, 153, 154, 155, 158, 159, 160, 161, 162, 163, 164, 165, 166, 169, 170, 171, 173, 174, 175, 176, 178, 180, 182, 183, 184, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197, 200, 201, 202, 203, 204, 205, 206, 207, 209, 210, 211, 212, 213, 214, 215, 218, 219, 220, 222, 223, 224, 225, 226, 229, 232, 234, 235, 236, 237, 238, 241, 242, 243, 244, 246, 249, 250, 251, 252, 253, 255, 256, 258, 260, 261, 262, 264, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 280, 281, 282

/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Completed Fold 0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 48, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 63, 64, 65, 66, 67, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 88, 89, 91, 93, 94, 96, 97, 98, 101, 102, 104, 105, 112, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 140, 141, 143, 144, 145, 146, 147, 148, 149, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 167, 168, 170, 171, 172, 173, 176, 177, 178, 179, 181, 182, 185, 186, 187, 189, 190, 193, 194, 196, 197, 198, 199, 200, 201, 202, 204, 206, 207, 208, 209, 213, 215, 216, 217, 221, 222, 223, 224, 225, 226, 227, 228, 230, 231, 232, 233, 234, 235, 236, 237, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 271, 273, 275, 276, 277, 2

/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Completed Fold 1
[1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 53, 54, 55, 57, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 83, 84, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 98, 99, 100, 101, 102, 103, 105, 106, 107, 108, 109, 110, 111, 113, 114, 116, 117, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 132, 133, 134, 135, 137, 139, 140, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 156, 157, 158, 160, 161, 163, 164, 165, 166, 167, 168, 169, 171, 172, 174, 175, 177, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 195, 196, 197, 198, 199, 200, 201, 202, 203, 205, 206, 207, 208, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 247, 248, 249, 250, 251, 252, 253, 254, 2

/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Completed Fold 2
[0, 3, 4, 5, 6, 7, 8, 9, 12, 14, 15, 16, 17, 18, 19, 23, 24, 25, 26, 29, 30, 31, 32, 33, 34, 37, 39, 41, 43, 44, 45, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 75, 76, 77, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 141, 142, 143, 144, 146, 149, 150, 151, 152, 154, 155, 156, 157, 159, 160, 161, 162, 164, 166, 167, 168, 169, 170, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212, 214, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 227, 228, 229, 230, 231, 232, 233, 234, 235, 238, 239, 240, 241, 242, 245, 246, 247, 248, 249, 251, 252, 253, 254, 255, 256, 257, 258, 259, 26

/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Completed Fold 3
[0, 1, 2, 6, 7, 8, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 56, 57, 58, 59, 60, 61, 62, 63, 65, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 118, 120, 121, 122, 123, 124, 128, 129, 131, 132, 134, 135, 136, 138, 139, 140, 141, 142, 144, 145, 147, 148, 149, 150, 151, 152, 153, 155, 156, 157, 158, 159, 162, 163, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 191, 192, 193, 194, 195, 196, 198, 199, 203, 204, 205, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 226, 227, 228, 229, 230, 231, 233, 234, 236, 237, 238, 239, 240, 243, 244, 245, 246, 247, 248, 250, 251, 252, 254, 256, 257, 258, 259, 261, 

/Users/haydenraikes/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [51]:

cv = KFold(n_splits=5, shuffle=True, random_state=42)

r2Scores = []
rmseScores = []

for i, (train_index, test_index) in enumerate(cv.split(xDF, yDF)):

    ### making training and validation sets
    # Convert indices to list
    train_index = train_index.tolist()
    test_index = test_index.tolist()
    
    # Split the data into training and testing sets for this fold
    xTrain, xTest = xDF.iloc[train_index], xDF.iloc[test_index]
    yTrain, yTest = yDF.iloc[train_index], yDF.iloc[test_index]

    ### feature scaling
    xScaler = StandardScaler()
    xColNames = xTrain.columns.values.tolist()
    # train the scaler and apply it to the training set
    xTrainScaled = xScaler.fit_transform(xTrain[xColNames])
    # apply the scaling to the testing set
    xTestScaled = xScaler.transform(xTest[xColNames])

    ### model training
    # instantiate the model
    clf = AdaBoostRegressor(**adaBoostParams)
    # Train the classifier on the training data
    clf.fit(xTrain, yTrain)
    
    ### model prediction and evaluation
    # Make predictions on the test data
    y_pred = clf.predict(xTest)
    
    # Calculate metrics and store them
    r2Score = r2_score(yTest, y_pred)
    r2Scores.append(r2Score)

    rmseScore = mean_squared_error(yTest, y_pred, squared=False)
    rmseScores.append(rmseScore)

    print(f"Completed Fold {i}")

### Calculate the mean scores across all folds
avgR2Score = sum(r2Scores) / len(r2Scores)
print("Mean r squared score:", avgR2Score)

avgRMSE = sum(rmseScores) / len(rmseScores)
print("Mean rmse:", avgRMSE)

Completed Fold 0
Completed Fold 1
Completed Fold 2
Completed Fold 3
Completed Fold 4
Mean r squared score: 0.3756808157591452
Mean rmse: 0.056051040132237016


In [53]:

cv = KFold(n_splits=5, shuffle=True, random_state=42)

r2Scores = []
rmseScores = []

for i, (train_index, test_index) in enumerate(cv.split(xDF, yDF)):

    ### making training and validation sets
    # Convert indices to list
    train_index = train_index.tolist()
    test_index = test_index.tolist()
    
    # Split the data into training and testing sets for this fold
    xTrain, xTest = xDF.iloc[train_index], xDF.iloc[test_index]
    yTrain, yTest = yDF.iloc[train_index], yDF.iloc[test_index]

    ### feature scaling
    xScaler = StandardScaler()
    xColNames = xTrain.columns.values.tolist()
    # train the scaler and apply it to the training set
    xTrainScaled = xScaler.fit_transform(xTrain[xColNames])
    # apply the scaling to the testing set
    xTestScaled = xScaler.transform(xTest[xColNames])

    ### model training
    # instantiate the model
    clf = RandomForestRegressor(**randomForestParams)
    # Train the classifier on the training data
    clf.fit(xTrain, yTrain)
    
    ### model prediction and evaluation
    # Make predictions on the test data
    y_pred = clf.predict(xTest)
    
    # Calculate metrics and store them
    r2Score = r2_score(yTest, y_pred)
    r2Scores.append(r2Score)

    rmseScore = mean_squared_error(yTest, y_pred, squared=False)
    rmseScores.append(rmseScore)

    print(f"Completed Fold {i}")

### Calculate the mean scores across all folds
avgR2Score = sum(r2Scores) / len(r2Scores)
print("Mean r squared score:", avgR2Score)

avgRMSE = sum(rmseScores) / len(rmseScores)
print("Mean rmse:", avgRMSE)

Completed Fold 0
Completed Fold 1
Completed Fold 2
Completed Fold 3
Completed Fold 4
Mean r squared score: 0.615180148487025
Mean rmse: 0.04352888628807487


In [46]:

cv = KFold(n_splits=5, shuffle=True, random_state=42)

r2Scores = []
rmseScores = []

for i, (train_index, test_index) in enumerate(cv.split(xDF, yDF)):

    ### making training and validation sets
    # Convert indices to list
    train_index = train_index.tolist()
    test_index = test_index.tolist()
    
    # Split the data into training and testing sets for this fold
    xTrain, xTest = xDF.iloc[train_index], xDF.iloc[test_index]
    yTrain, yTest = yDF.iloc[train_index], yDF.iloc[test_index]

    ### feature scaling
    xScaler = StandardScaler()
    xColNames = xTrain.columns.values.tolist()
    # train the scaler and apply it to the training set
    xTrainScaled = xScaler.fit_transform(xTrain[xColNames])
    # apply the scaling to the testing set
    xTestScaled = xScaler.transform(xTest[xColNames])

    ### model training
    # instantiate the model
    clf = Ridge(**ridgeParams)
    # Train the classifier on the training data
    clf.fit(xTrain, yTrain)
    
    ### model prediction and evaluation
    # Make predictions on the test data
    y_pred = clf.predict(xTest)
    
    # Calculate metrics and store them
    r2Score = r2_score(yTest, y_pred)
    r2Scores.append(r2Score)

    rmseScore = mean_squared_error(yTest, y_pred, squared=False)
    rmseScores.append(rmseScore)

    print(f"Completed Fold {i}")

### Calculate the mean scores across all folds
avgR2Score = sum(r2Scores) / len(r2Scores)
print("Mean r squared score:", avgR2Score)

avgRMSE = sum(rmseScores) / len(rmseScores)
print("Mean rmse:", avgRMSE)

Completed Fold 0
Completed Fold 1
Completed Fold 2
Completed Fold 3
Completed Fold 4
Mean r squared score: 0.4490894326754554
Mean rmse: 0.05287387633320041


In [47]:

cv = KFold(n_splits=5, shuffle=True, random_state=42)

r2Scores = []
rmseScores = []

for i, (train_index, test_index) in enumerate(cv.split(xDF, yDF)):

    ### making training and validation sets
    # Convert indices to list
    train_index = train_index.tolist()
    test_index = test_index.tolist()
    
    # Split the data into training and testing sets for this fold
    xTrain, xTest = xDF.iloc[train_index], xDF.iloc[test_index]
    yTrain, yTest = yDF.iloc[train_index], yDF.iloc[test_index]

    ### feature scaling
    xScaler = StandardScaler()
    xColNames = xTrain.columns.values.tolist()
    # train the scaler and apply it to the training set
    xTrainScaled = xScaler.fit_transform(xTrain[xColNames])
    # apply the scaling to the testing set
    xTestScaled = xScaler.transform(xTest[xColNames])

    ### model training
    # instantiate the model
    clf = Lasso(**lassoParams)
    # Train the classifier on the training data
    clf.fit(xTrain, yTrain)
    
    ### model prediction and evaluation
    # Make predictions on the test data
    y_pred = clf.predict(xTest)
    
    # Calculate metrics and store them
    r2Score = r2_score(yTest, y_pred)
    r2Scores.append(r2Score)

    rmseScore = mean_squared_error(yTest, y_pred, squared=False)
    rmseScores.append(rmseScore)

    print(f"Completed Fold {i}")

### Calculate the mean scores across all folds
avgR2Score = sum(r2Scores) / len(r2Scores)
print("Mean r squared score:", avgR2Score)

avgRMSE = sum(rmseScores) / len(rmseScores)
print("Mean rmse:", avgRMSE)

Completed Fold 0
Completed Fold 1
Completed Fold 2
Completed Fold 3
Completed Fold 4
Mean r squared score: 0.04901804127024867
Mean rmse: 0.06975249925180235


In [48]:

cv = KFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {'fit_intercept': [True, False], 'normalize': [True, False]}

r2Scores = []
rmseScores = []

for i, (trainIndex, testIndex) in enumerate(cv.split(xDF, yDF)):
    #training and validation sets
    #convert indices to list
    trainIndex = trainIndex.tolist()
    testIndex = testIndex.tolist()

    print(trainIndex)

    #split the data into training and testing sets for this fold
    xTrain, xTest = xDF.iloc[trainIndex], xDF.iloc[testIndex]
    yTrain, yTest = yDF.iloc[trainIndex], yDF.iloc[testIndex]
     

    #feature scaling 
    ### feature scaling
    xScaler = StandardScaler()
    xColNames = xTrain.columns.values.tolist()
    # train the scaler and apply it to the training set
    xTrainScaled = xScaler.fit_transform(xTrain[xColNames])
    # apply the scaling to the testing set
    xTestScaled = xScaler.transform(xTest[xColNames])

    ### model training
    # instantiate the model
    clf = ElasticNet(**elasticNetParams)
    # Train the classifier on the training data
    clf.fit(xTrainScaled, yTrain)
    
    ### model prediction and evaluation
    # Make predictions on the test data
    yPred = clf.predict(xTestScaled)
    
    # Calculate metrics and store them
    r2Score = r2_score(yTest, yPred)
    r2Scores.append(r2Score)

    rmseScore = mean_squared_error(yTest, yPred, squared=False)
    rmseScores.append(rmseScore)

    print(f"Completed Fold {i}")

### Calculate the mean scores across all folds
avgR2Score = sum(r2Scores) / len(r2Scores)
print("Mean r squared score:", avgR2Score)

avgRMSE = sum(rmseScores) / len(rmseScores)
print("Mean rmse:", avgRMSE)



[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 16, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 32, 34, 35, 36, 38, 39, 40, 42, 43, 46, 47, 48, 49, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 81, 82, 83, 85, 86, 87, 89, 90, 92, 94, 95, 97, 98, 99, 100, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 123, 125, 126, 127, 128, 129, 130, 131, 133, 134, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 147, 148, 150, 151, 152, 153, 154, 155, 158, 159, 160, 161, 162, 163, 164, 165, 166, 169, 170, 171, 173, 174, 175, 176, 178, 180, 182, 183, 184, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197, 200, 201, 202, 203, 204, 205, 206, 207, 209, 210, 211, 212, 213, 214, 215, 218, 219, 220, 222, 223, 224, 225, 226, 229, 232, 234, 235, 236, 237, 238, 241, 242, 243, 244, 246, 249, 250, 251, 252, 253, 255, 256, 258, 260, 261, 262, 264, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 280, 281, 282